In [34]:
import mne
import pathlib
import json
import re
import numpy as np

In [48]:
USEFUL_CHANNELS = [
"FP1-F7",
"F7-T7",
"T7-P7",
"P7-O1",
"FP1-F3",
"F3-C3",
"C3-P3",
"P3-O1",
"FP2-F4",
"F4-C4",
"C4-P4",
"P4-O2",
"FP2-F8",
"F8-T8",
"T8-P8",
"P8-O2",
"FZ-CZ",
"CZ-PZ",
"P7-T7",
"T7-FT9",
"FT9-FT10",
"FT10-T8",
"T8-P8",
]

# new_names = dict(
#     (ch_name,
#     ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
#     for ch_name in USEFUL_CHANNELS)
# USEFUL_CHANNELS = [new_names[x] for x in USEFUL_CHANNELS]

In [72]:
MISSING = ['O1-P7', 'O2-P4', 'F8-FP2', 'F7-T7', 'F4-FP2', 'CZ-PZ', 'O1-P3', 'CZ-FZ', 'F3-FP1', 'F8-T8', 'C3-P3', 'F7-FP1', 'O2-P8', 'P8-T8', 'C3-F3', 'C4-P4', 'C4-F4']

In [55]:
path = pathlib.Path("/media/HDD/Unisa/Datasets/CHB_MIT/chb12/chb12_27.edf")

raw = mne.io.read_raw_edf(path)

raw.ch_names
# for ch in MISSING:
#     for found_ch in raw.ch_names:
#         if ch in found_ch:
#             print(f"{ch} ~ {found_ch}")
# data = raw.get_data()
    
# channel1 = data[raw.ch_names.index("T8-P8-0"), :]
# channel2 = data[raw.ch_names.index("T8-P8-1"), :]


# new_names = dict(
#     (ch_name,
    # #  ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
#     # for ch_name in raw.ch_names)
# # raw.rename_channels(new_names)

# # new_names = dict(
#     # (ch_name,
    # #  ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
#     # for ch_name in USEFUL_CHANNELS)
# USEFUL_CHANNELS = [new_names[x] for x in USEFUL_CHANNELS]

Extracting EDF parameters from /media/HDD/Unisa/Datasets/CHB_MIT/chb12/chb12_27.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_5567/3143991756.py:3: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(path)
/tmp/ipykernel_5567/3143991756.py:3: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(path)


['F7-CS2',
 'T7-CS2',
 'P7-CS2',
 '--0',
 'FP1-CS2',
 'F3-CS2',
 'C3-CS2',
 'P3-CS2',
 'O1-CS2',
 '--1',
 'FZ-CS2',
 'CZ-CS2',
 'PZ-CS2',
 '--2',
 'FP2-CS2',
 'F4-CS2',
 'C4-CS2',
 'P4-CS2',
 'O2-CS2',
 '--3',
 'F8-CS2',
 'T8-CS2',
 'P8-CS2',
 '--4',
 'C2-CS2',
 'C6-CS2',
 'CP2-CS2',
 'CP4-CS2',
 'CP6-CS2']

In [12]:
montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage, on_missing="ignore")



<RawEDF | chb15_01.edf, 31 x 922112 (3602.0 s), ~36 kB, data not loaded>

In [20]:
print("raw contains", len(raw.ch_names))

print("Missing")
for channel in USEFUL_CHANNELS:
    if channel not in raw.ch_names:
        print(channel)

print("\nExtra")
for channel in raw.ch_names:
    if channel not in USEFUL_CHANNELS:
        print(channel)

raw contains 31
Missing
P7-T7
T7-FT9
FT9-FT10
FT10-T8

Extra
--0
--1
--2
--3
--4
FC1-REF
FC2-REF
FC5-REF
FC6-REF
CP1-REF
CP2-REF
CP5-REF
CP6-REF


In [83]:
montage = mne.channels.make_standard_montage("standard_1005")

# exclude = ["(?i).*-REF", ".*-Ref", "ECG", "VNS"]
# for channel in COMMON_CHANNELS.split(","):
#     for i in range(5):
#         exclude.append(f"{channel}-{i+1}")

def read_data(path):
    raw = mne.io.read_raw_edf(path, stim_channel=None, include=COMMON_CHANNELS, verbose="ERROR")

    if "T8-P8-1" in raw.ch_names:
        raw.drop_channels("T8-P8-1")
        new_names = {"T8-P8-0": "T8-P8"}
        raw.rename_channels(new_names)

    raw.reorder_channels(COMMON_CHANNELS)


    # new_names = dict(
    #     (ch_name,
    #     ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
    #     for ch_name in raw.ch_names)
    # raw.rename_channels(new_names)

    # raw = raw.set_montage(montage, on_missing="ignore")

    # new_ch_names = raw.ch_names
    return raw.ch_names

    # if len(new_ch_names) < len(original_ch_names):
    #     raise f"Canali diminuiti in {path}"

    # return raw
    # parsed_ch_names = sorted(set(["-".join(channel.split("-")[:2]) for channel in original_ch_names if not channel.startswith("--")]))
    # return ",".join(parsed_ch_names)

In [73]:
COMMON_CHANNELS = [
    "FP1-F7",
    "F7-T7",
    "T7-P7",
    "P7-O1",
    "FP1-F3",
    "F3-C3",
    "C3-P3",
    "P3-O1",
    "FP2-F4",
    "F4-C4",
    "C4-P4",
    "P4-O2",
    "FP2-F8",
    "F8-T8",
    "P8-O2",
    "FZ-CZ",
    "CZ-PZ",
    "T8-P8"
]
# COMMON_CHANNELS = ["-".join(sorted(channel.split("-")[:2])) for channel in COMMON_CHANNELS]

In [50]:
slices_path = pathlib.Path("output/300/slices.json")
dataset_path = pathlib.Path("/media/HDD/Unisa/Datasets/CHB_MIT")
with open(slices_path) as f:
    data = json.load(f)


dirs = (path for path in dataset_path.iterdir() if path.is_dir())

# for directory in dirs:
#     for edfpath in (edfpath for edfpath in directory.iterdir() if edfpath.name.endswith("edf")):
#         minimum = read_data(edfpath)
#         channels.add(minimum)

count = 563

# filter_out_regex = ["(?i).*-REF", "ECG", "VNS", ".*--.*"]
filter_out_regex = ["(?i).*-REF", ".*--.*", "ECG", "VNS"]

every_channel = set()
for patient, files_data in data.items():
    for filename, slices in files_data.items():
        edf_path = dataset_path.joinpath(patient, filename)
        # raw = read_data(path)
        channels = read_data(edf_path)
        for channel in channels:
            if not any(re.match(regex, channel) for regex in filter_out_regex):
                every_channel.add(channel)

print(every_channel)
print(f"{len(every_channel)=}")

missing_channels = set()
for channel in COMMON_CHANNELS:
    if channel not in every_channel:
        missing_channels.add(channel)

print(f"{missing_channels=}")

for channel in missing_channels:
    for found_channel in every_channel:
        if channel in found_channel:
            print(f"{channel} in {found_channel}")
        # if minimum != COMMON_CHANNELS:
        #     print(f"{edf_path} {minimum}")
        # channels.add(minimum)

# print(channels)

        

{'.-3', 'C6', 'CP2', 'P7-O1', 'T8-P8-1', 'F4', 'FP2', 'C4-CS2', 'O2', 'F7-T7', 'P7', 'C6-CS2', 'CP2-CS2', 'FZ', 'F3-C3', 'PZ', 'CP6-CS2', '.-0', 'O1-CS2', 'C4-P4', 'P8', 'P3-CS2', '.-4', '.-1', 'C4', 'CP4-CS2', 'FP2-CS2', 'T7', 'EKG1-CHIN', 'P8-O2', 'F4-CS2', 'F7', 'FT10-T8', 'C2', 'CP6', 'P3', 'FZ-CS2', '01', 'F7-CS2', 'F8', 'CZ-CS2', 'F8-T8', 'FP1-F3', 'T8-CS2', 'CP4', 'C3', 'T8', 'C3-P3', 'LUE-RAE', 'FP1', 'FT9-FT10', 'P7-T7', 'CZ-PZ', 'O2-CS2', 'P7-CS2', 'T7-P7', 'EKG1-EKG2', 'T7-CS2', 'FZ-CZ', 'C3-CS2', 'F8-CS2', 'T7-FT9', 'FP1-F7', 'F4-C4', 'PZ-CS2', 'CZ', 'P3-O1', 'FP2-F8', '.-2', 'P4-CS2', 'T8-P8-0', 'F3', 'T8-P8', 'F3-CS2', 'PZ-OZ', 'P4-O2', 'LOC-ROC', 'FP2-F4', 'C2-CS2', 'FP1-CS2', 'P4', 'P8-CS2'}
len(every_channel)=82
missing_channels={'O1-P7', 'O2-P4', 'F8-FP2', 'F4-FP2', 'O1-P3', 'CZ-FZ', 'F3-FP1', 'F7-FP1', 'O2-P8', 'P8-T8', 'C3-F3', 'C4-F4'}


In [51]:
slices_path = pathlib.Path("output/300/slices.json")
dataset_path = pathlib.Path("/media/HDD/Unisa/Datasets/CHB_MIT")
with open(slices_path) as f:
    data = json.load(f)


dirs = (path for path in dataset_path.iterdir() if path.is_dir())

# for directory in dirs:
#     for edfpath in (edfpath for edfpath in directory.iterdir() if edfpath.name.endswith("edf")):
#         minimum = read_data(edfpath)
#         channels.add(minimum)

count = 563

# filter_out_regex = ["(?i).*-REF", "ECG", "VNS", ".*--.*"]
filter_out_regex = ["(?i).*-REF", ".*--.*", "ECG", "VNS"]

every_channel = set()
for patient, files_data in data.items():
    for filename, slices in files_data.items():
        edf_path = dataset_path.joinpath(patient, filename)
        # raw = read_data(path)
        channels = read_data(edf_path)
        actual_channels = set()
        for channel in channels:
            channel = "-".join(sorted(channel.split("-")[:2]))
            if not any(re.match(regex, channel) for regex in filter_out_regex):
                actual_channels.add(channel)
        missing_channels = set()
        for channel in COMMON_CHANNELS:
            if channel not in actual_channels:
                missing_channels.add(channel)
        
        if len(missing_channels):
            print(f"{missing_channels=} {edf_path=}")

            for channel in missing_channels:
                for found_channel in actual_channels:
                    if channel in found_channel:
                        print(f"{channel} in {found_channel}")

missing_channels={'O1-P7', 'O2-P4', 'F8-FP2', 'F7-T7', 'F4-FP2', 'CZ-PZ', 'O1-P3', 'CZ-FZ', 'F3-FP1', 'F8-T8', 'C3-P3', 'F7-FP1', 'O2-P8', 'P8-T8', 'P7-T7', 'C3-F3', 'C4-P4', 'C4-F4'} edf_path=PosixPath('/media/HDD/Unisa/Datasets/CHB_MIT/chb12/chb12_27.edf')
missing_channels={'O1-P7', 'O2-P4', 'F8-FP2', 'F7-T7', 'F4-FP2', 'CZ-PZ', 'O1-P3', 'CZ-FZ', 'F3-FP1', 'F8-T8', 'C3-P3', 'F7-FP1', 'O2-P8', 'P8-T8', 'P7-T7', 'C3-F3', 'C4-P4', 'C4-F4'} edf_path=PosixPath('/media/HDD/Unisa/Datasets/CHB_MIT/chb12/chb12_28.edf')
missing_channels={'O1-P7', 'O2-P4', 'F8-FP2', 'F7-T7', 'F4-FP2', 'CZ-PZ', 'O1-P3', 'CZ-FZ', 'F3-FP1', 'F8-T8', 'C3-P3', 'F7-FP1', 'O2-P8', 'P8-T8', 'P7-T7', 'C3-F3', 'C4-P4', 'C4-F4'} edf_path=PosixPath('/media/HDD/Unisa/Datasets/CHB_MIT/chb12/chb12_29.edf')


In [60]:
DISCARDED_EDFS = [
    "chb12_27.edf",
    "chb12_28.edf",
    "chb12_29.edf",
]

In [84]:
slices_path = pathlib.Path("output/300/slices.json")
dataset_path = pathlib.Path("/media/HDD/Unisa/Datasets/CHB_MIT")
with open(slices_path) as f:
    data = json.load(f)


dirs = (path for path in dataset_path.iterdir() if path.is_dir())

channel_configurations = set()
channel_amounts = set()
for patient, files_data in data.items():
    for filename, slices in files_data.items():
        edf_path = dataset_path.joinpath(patient, filename)
        if edf_path.name not in DISCARDED_EDFS:
            # raw = read_data(path)
            channels = read_data(edf_path)
            channel_configurations.add(tuple(channels))
            if len(channels) == 20:
                pass
            else:
                channel_amounts.add(len(channels))

for conf in channel_configurations:
    for ch in COMMON_CHANNELS:
        if ch not in conf:
            print(conf, ch)

# print(channel_amounts)
print(f"{channel_configurations=}")

In [43]:
mne.channels.get_builtin_montages()

['standard_1005',
 'standard_1020',
 'standard_alphabetic',
 'standard_postfixed',
 'standard_prefixed',
 'standard_primed',
 'biosemi16',
 'biosemi32',
 'biosemi64',
 'biosemi128',
 'biosemi160',
 'biosemi256',
 'easycap-M1',
 'easycap-M10',
 'EGI_256',
 'GSN-HydroCel-32',
 'GSN-HydroCel-64_1.0',
 'GSN-HydroCel-65_1.0',
 'GSN-HydroCel-128',
 'GSN-HydroCel-129',
 'GSN-HydroCel-256',
 'GSN-HydroCel-257',
 'mgh60',
 'mgh70',
 'artinis-octamon',
 'artinis-brite23',
 'brainproducts-RNP-BA-128']